## This notebook pulls pricing information from Coinmarketcap $$$ 

### Data will consist of information from ranks, coins, stats, new coins, trends, and events

In [42]:
from pymarketcap import Pymarketcap
import re
import json
import pprint
import pandas as pd
import requests
from datetime import datetime, timedelta 
from bs4 import BeautifulSoup
import urllib3
import time
import logging
import numpy as np
from apscheduler.schedulers.blocking import BlockingScheduler
import re
import urllib.request
import numpy as np
pd.options.display.float_format = '{:20,.2f}'.format

##### Defines a function that allows for replacement for new coin date added

In [43]:
def setdate(x):
    if len(x) == 5 or len(x) == 7:
        return datetime.today().strftime("%m/%d/%Y")
    elif len(x) == 8:
        newDate = datetime.today() - timedelta(days=int(x[:1]))
        return newDate.strftime("%m/%d/%Y")
    else:
        newDate = datetime.today() - timedelta(days=int(x[:2]))
        return newDate.strftime("%m/%d/%Y")

###### Sets variable coinmarketcap with Pymarketcap API/Scraper 

In [44]:
from pymarketcap import Pymarketcap
coinmarketcap = Pymarketcap()
symbols = coinmarketcap.symbols

###### Set initial Date/Time.. Will be setting throughout 

In [45]:
#Pulls date and time information
moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())
today = pd.Timestamp("today").strftime("%m/%d/%Y")
hh = pd.Timestamp("today").strftime("%H")
mm = pd.Timestamp("today").strftime("%M")

###### Coinmarketcap statistics - up to date 1/14/2018

In [47]:
try:
    stats = coinmarketcap.stats()
    stats = pd.DataFrame(stats, index=[0])
    
    stats['Date'] = today
    stats['Hour'] = hh
    stats['Minute'] = mm
    stats['Now'] = pd.Timestamp("today")
    
    stats.to_csv('Coinmarketcap/boom/stats/stats_'+moment+'.csv', sep=',') 
    print("Look at all those stats 0.o")
except:
    print("*NO STATS GATHERED*")

Look at all those stats 0.o


###### Coin Data - Updated 1/14/2018

In [49]:
try:
    from pymarketcap import Pymarketcap
    coinmarketcap = Pymarketcap()
    ticker = coinmarketcap.ticker(limit=1500)
    ticker = pd.DataFrame(ticker)

    ticker['Date'] = today
    ticker['Hour'] = hh
    ticker['Minute'] = mm
    ticker['Now'] = pd.Timestamp("today")

    ticker.to_csv('Coinmarketcap/boom/coins/coins_'+moment+'.csv', sep=',')  
    print("Chaaaa-CHING! The coin data is in")
except:
    print("*NO COINS GATHERED* *YOU ARE LOSING OUT ON BAGS*")

Chaaaa-CHING! The coin data is in


###### Coin exchange info for each token - Updated 1/14/2018

In [ ]:
for coin2 in symbols:
    try:
        from pymarketcap import Pymarketcap
        coinmarketcap = Pymarketcap()
        markets = coinmarketcap.markets(coin2)
        markets = pd.DataFrame(markets['markets'])

        markets['Date'] = today
        markets['Hour'] = hh
        markets['Minute'] = mm
        markets['Now'] = pd.Timestamp("today")
        
        markets.to_csv('Coinmarketcap/boom/markets/markets_'+coin2+'_'+moment+'.csv', sep=',') 
    except:
        print("No market data was captured for ", coin2)
        pass
print("I hear the exchange trades from here.. exchange data collected :)")

###### Gainers and Losers (1h, 24h, 7d) - Updated 1/14/2018

###### ******Currently no 7d Gainers being captured******

###### New coins - Updated 1/14/2018

In [102]:
#Set date and time
today = pd.Timestamp("today").strftime("%m/%d/%Y")
hh = pd.Timestamp("today").strftime("%H")
mm = pd.Timestamp("today").strftime("%M")
moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())

data = coinmarketcap.recently()
data = pd.DataFrame(data)

data.head()

data['Date'] = today
data['Hour'] = hh
data['Minute'] = mm 
data['days_ago']= data['added'].str.extract('(\d.)', expand=True)
data['Now'] = pd.Timestamp("today")
data['days_ago'] = data['days_ago'].apply(pd.to_numeric)
data['Date_Added'] = data['Now'] - pd.to_timedelta(data['days_ago'], unit='d')
data['Date_Added'] = data['Date_Added'].dt.date

#Reimport due to naming issues
from pymarketcap import Pymarketcap
coinmarketcap = Pymarketcap()


#Get Market date for new coins
for coin2 in data["symbol"]:
    try:
        markets = coinmarketcap.markets(coin2)
        markets = pd.DataFrame(markets['markets'])


        #add 'Today' 'Hour' and 'minutes' column
        markets['Date'] = today
        markets['Hour'] = hh
        markets['Minute'] = mm
        markets['Now'] = pd.Timestamp("today")

        #Save CSV
        markets.to_csv('Coinmarketcap/boom/markets/new/markerts_'+coin2+'_'+moment+'.csv', sep=',') 
        print("Market info for new coin", coin2," captured!")
    except:
        print("************ERROR COIN", coin2, "DATA NOT CAPUTRED*************")
        pass
        
        
    ###Upload datafile

    
data.to_csv('Coinmarketcap/boom/new/new_'+moment+'.csv', sep=',') 
print("Gotta catch all the new coins!")

Market info for new coin LNC  captured!
Market info for new coin PHI  captured!
Market info for new coin DOCK  captured!
Market info for new coin MSR  captured!
Market info for new coin CHX  captured!
Market info for new coin SRCOIN  captured!
Market info for new coin CROP  captured!
************ERROR COIN PXC DATA NOT CAPUTRED*************
Market info for new coin MRK  captured!
Market info for new coin EARTH  captured!
Market info for new coin FTX  captured!
Market info for new coin EFX  captured!
Market info for new coin DERO  captured!
************ERROR COIN CRC DATA NOT CAPUTRED*************
Market info for new coin KST  captured!
Market info for new coin XHV  captured!
Market info for new coin ADI  captured!
Market info for new coin DIG  captured!
Market info for new coin MFG  captured!
Market info for new coin BRM  captured!
Market info for new coin DAN  captured!
Market info for new coin XMC  captured!
Market info for new coin BUBO  captured!
Market info for new coin AUC  captu

###### Exchange Data

In [ ]:
try:
    exchanges = coinmarketcap.exchanges(limit=200)
    exchanges = pd.DataFrame(exchanges)

    exchanges['Date'] = today
    exchanges['Hour'] = hh
    exchanges['Minute'] = mm
    exchanges['Now'] = pd.Timestamp("today")
    del exchanges['markets']

    exchanges.to_csv('Coinmarketcap/exchanges/exchanges_'+moment+'.csv', sep=',')  
    print("Exchange data collected!")
except:
    print("TETHER!!! ITS A LIE!!! EXCHANGE DATA WASN'T COLLECTED AND IT IS ALL GOING TO 0!!!!")

         
for exchange in exchanges['name']:
    try:
        exchange_data = coinmarketcap.exchange(exchange)
        exchange_data = pd.DataFrame(exchange_data)
        
        exchange_data['Date'] = today
        exchange_data['Hour'] = hh
        exchange_data['Minute'] = mm
        exchange_data['Now'] = pd.Timestamp("today")
          
        exchanges.to_csv('Coinmarketcap/exchanges/exchange_data_'+exchange+'_'+moment+'.csv', sep=',') 
    except:
          print("No exchange data was collected for ", exchange)
          pass
print("Look at all those listings on exchagnes :)")
    
    
    


In [ ]:
exchanges

In [ ]:
for i in range(1,10):
    print(i)

In [ ]:
from pymarketcap import Pymarketcap
import re
import json
import pprint
import pandas as pd
import requests
from datetime import datetime, timedelta 
from bs4 import BeautifulSoup
import urllib3
import time
import logging
import numpy as np
from apscheduler.schedulers.blocking import BlockingScheduler
import re
import urllib.request
import numpy as np
pd.options.display.float_format = '{:20,.2f}'.format

x = 0
y = 0

df = pd.read_csv('Coinmarketcap/CoinCal/Events.csv', index_col=False, header=0);
df = pd.DataFrame(df)
sheet = []
for page in range(1,200):
    try:
        for link in links:
                print("Coinmarketcal page "+str(page)+" in progress...")
                text = urllib.request.urlopen('https://coinmarketcal.com/?page='+str(page)).read().decode()
                moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())


                coins = re.findall('(?<=data-coin=").+(?<=\))',text)

                symbol = []
                dates = re.findall('(?<=data-date=").+(?=" d)',text)
                event = re.findall('(?<=data-title=").+(?="><i)',text)
                date_added = re.findall('(?<=<p class="added-date">\(Added ).+(?=\)</p>)',text)
                certainty = re.findall('(?<=aria-valuenow=").+?(?=" role)', text)
                for token in coins:
                    start = token.find( '(' )
                    end = token.find( ')' )
                    if start != -1 and end != -1:
                        symbol.append(token[start+1:end])

                for i in range(len(coins)):
                    if ((df['Symbol'] == str(symbol[i])) & (df['Overview'] == str(event[i]))).any():
                        x += 1
                        if ((df['Symbol'] == str(symbol[i])) & (df['Overview'] == str(event[i])) & (df['Certainty'] != float(certainty[i]))).any():
                            df.loc[((df['Symbol'] == str(symbol[i])) & (df['Overview'] == str(event[i]))), "Certainty"] = certainty[i]
                    else:
                        sheet.append([str(coins[i]),str(symbol[i]), str(dates[i]),  str(event[i]), float(certainty[i]), str(date_added[i])])
                        y += 1
                        
    except:
        break


z = x+y
if sheet:
    data = pd.DataFrame(sheet, columns = ["Name","Symbol", "Date", "Overview","Certainty","date_added"])
    new_df = df.append(data)
    new_df.to_csv('Coinmarketcap/CoinCal/Events.csv', header=True, sep=',', index = FALSE) 
print("Coinmarketcal Complete. " + str(y) + " of " + str(z) + " added")
 

In [ ]:
sheet = []
for page in range(1,200):
    try:
        for link in links:
                print("Coinmarketcal page "+str(page)+" in progress...")
                text = urllib.request.urlopen('https://coinmarketcal.com/pastevents?page='+str(page)).read().decode()
                moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())


                coins = re.findall('(?<=data-coin=").+(?<=\))',text)

                symbol = []
                dates = re.findall('(?<=data-date=").+(?=" d)',text)
                event = re.findall('(?<=data-title=").+(?="><i)',text)
                date_added = re.findall('(?<=<p class="added-date">\(Added ).+(?=\)</p>)',text)
                certainty = re.findall('(?<=aria-valuenow=").+?(?=" role)', text)
                for token in coins:
                    start = token.find( '(' )
                    end = token.find( ')' )
                    if start != -1 and end != -1:
                        symbol.append(token[start+1:end])

                for i in range(len(coins)):
                        sheet.append([str(coins[i]),str(symbol[i]), str(dates[i]),  str(event[i]), float(certainty[i]), str(date_added[i])])

    except:
        break



data = pd.DataFrame(sheet, columns = ["Name","Symbol", "Date", "Overview","Certainty","date_added"])
data.to_csv('Coinmarketcap/CoinCal/PastEvents.csv', header=True, sep=',') 
print("Coinmarketcal Complete")


##### Past events

In [ ]:
import re
import urllib.request
import pandas as pd
import numpy as np

exchangesURL = 'https://coinmarketcal.com/pastevents?page=1'

allEvents = []
allDescription = []
text = urllib.request.urlopen(exchangesURL).read().decode()
coins = re.findall('(?<=<h5><strong>).+(?=</strong>)', text)
descriptions = re.findall('(?<=<h5>).+(?=\n                            <span>)', text, re.DOTALL)
#allEvents += coins
#allDescription += descriptions
#return allEvents



for i in range(len(coins)):
    if i % 2 == 0:
        dates.append(coins[i])
    else:
        tokens.append(coins[i])
eventDates = []
for i in range(len(dates)):
    if i % 2 == 0:
        eventDates.append(dates[i])

len(descriptions)

In [ ]:
text

In [ ]:
descriptions

In [ ]:
today = pd.Timestamp("today").strftime("%m/%d/%Y")
hh = pd.Timestamp("today").strftime("%H")
mm = pd.Timestamp("today").strftime("%M")

for link in links:
    
    text = urllib.request.urlopen('http://coinmarketcal.com/').read().decode()
    moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())

    sheet = []
    coins = re.findall('(?<=<h5><strong>).+(?=</strong>)',text)
    dates = []
    tokens = []

    i = 0

    for i in range(len(coins)):
        if i % 2 == 0:
            dates.append(coins[i])
        else:
            tokens.append(coins[i])
    eventDates = []
    for i in range(len(dates)):
        if i % 2 == 0:
            eventDates.append(dates[i])

    symbol = []
    event = re.findall('(?<=h5>).+(?=\n                                                            )',text)
    description = re.findall('(?<=p class="description">).+?(?=</p>)',text, re.DOTALL)
    certainty = re.findall('(?<=aria-valuenow=").+?(?=" role)', text)
    for token in tokens:
        start = token.find( '(' )
        end = token.find( ')' )
        if start != -1 and end != -1:
            symbol.append(token[start+1:end])

    print("Coinmarketcal in progress...")

    parts = [coin.split() for coin in coins]
    for i in range(len(tokens)):
        if 'By' not in dates[i]:
            sheet.append([str(tokens[i]),str(symbol[i]), str(dates[i]), float(certainty[i]), str(event[i]), str(description[i])])
            print(sheet)
        else:
            sheet.append([str(tokens[i]),str(symbol[i]), str(dates[i][3:]), float(certainty[int(i)]), str(event[i]), str(description[i])])
            print(sheet)



    data = pd.DataFrame(sheet, columns = ["Name","Symbol", "Event", "Confidience","Title","Description"])



    data['Date'] = today
    data['Hour'] = hh
    data['Minute'] = mm
    markets['Now'] = pd.Timestamp("today")
    data.to_csv('Coinmarketcap/CoinCal/CC_'+moment+'.csv', header=True, sep=',') 
    print("Coinmarketcal Complete")


In [ ]:
2+2